# Module 6: Generators: Types

Adapted for UC Berkeley ELENG 194/294 from Chisel Bootcamp by ELENG 194/294 staff

Scala is a strongly-typed programming language.
This is a two-edged sword; on one hand, many programs that would compile and execute in Python (a dynamically-typed language) would fail at compile time in Scala.
On the other hand, programs that compile in Scala will contain many fewer runtime errors than a similar Python program.

In this section, our goal is to familiarize you with types in Scala.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# Static Types<a name="types-in-scala"></a>

## Types in Scala

All objects in Scala have a type, which is usually the object's class.
Let's see some:

In [ ]:
println(10.getClass)
println(10.0.getClass)
println("ten".getClass)

When you declare your own class, it has an associated type.

In [ ]:
class MyClass {
    def myMethod = ???
}
println(new MyClass().getClass)

While not required, it is HIGHLY recommended that you **define input and output types for all function declarations**.
This will let the Scala compiler catch improper use of a function.

In [ ]:
def double(s: String): String = s + s
// Uncomment the code below to test it
// double("hi")      // Proper use of double
// double(10)        // Bad input argument!
// double("hi") / 10 // Inproper use of double's output!

Functions that don't return anything return type `Unit`.

In [ ]:
var counter = 0
def increment(): Unit = {
    counter += 1
}
increment()

## Scala vs. Chisel Types<a name="scala-vs-chisel-types"></a>

Recap: Module 2.2 discussed the difference between Chisel types and Scala types, for example the fact that
```scala
val a = Wire(UInt(4.W))
a := 0.U
```
is legal because `0.U` is of type `UInt` (a Chisel type), whereas
```scala
val a = Wire(UInt(4.W))
a := 0
```
is illegal because 0 is type `Int` (a Scala type).

This is also true of `Bool`, a Chisel type which is distinct from `Boolean`.
```scala
val bool = Wire(Bool())
val boolean: Boolean = false
// legal
when (bool) { ... }
if (boolean) { ... }
// illegal
if (bool) { ... }
when (boolean) { ... }
```

If you make a mistake and mix up `UInt` and `Int` or `Bool` and `Boolean`, the Scala compiler will generally catch it for you.
This is because of Scala's static typing.
At compile time, the compiler is able to distinguish between Chisel and Scala types and also able to understand that `if ()` expects a `Boolean` and `when ()` expects a `Bool`.


## Scala Type Coercion<a name="type-coercion"></a>

<!-- typeOf. Scala has a function called `typeOf[T]` which returns a type object for `T`. -->
<!-- This doesn't actually seem useful to Chisel users... -->

### asInstanceOf

`x.asInstanceOf[T]` casts the object `x` to the type `T`. It throws an exception if the given object cannot be cast to type `T`.

In [ ]:
val x: UInt = 3.U
try {
  println(x.asInstanceOf[Int])
} catch {
  case e: java.lang.ClassCastException => println("As expected, we can't cast UInt to Int")
}

// But we can cast UInt to Data since UInt inherits from Data.
println(x.asInstanceOf[Data])


### Type Casting in Chisel

The code below will give an error if you try to run it without removing the comment.
What's the problem?
It is trying to assign a `UInt` to an `SInt`, which is illegal.

Chisel has a set of type casting functions.
The most general is `asTypeOf()`, which is shown below.
Some chisel objects also define `asUInt()` and `asSInt()` as well as some others.

If you remove the `//` from the code block below, the example should work for you.


In [ ]:
class TypeConvertDemo extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(4.W))
        val out = Output(SInt(4.W))
    })
    io.out := io.in//.asTypeOf(io.out)
}

test(new TypeConvertDemo) { c =>
      c.io.in.poke(3.U)
      c.io.out.expect(3.S)
      c.io.in.poke(15.U)
      c.io.out.expect(-1.S)
}

---
# Type Matching<a name="type-matching"></a>

## Match Operator
Recall that in 3.1 the match operator was introduced.
Type matching is especially useful when trying to write type-generic generators.
The following example shows an example of a "generator" that can add two literals of type `UInt` or `SInt`.
Later sections will talk more about writing type-generic generators.

**Note: there are much better and safer ways to write type-generic generators in Scala**.

In [ ]:
class ConstantSum(in1: Data, in2: Data) extends Module {
    val io = IO(new Bundle {
        val out = Output(chiselTypeOf(in1)) // in case in1 is literal then just get its type
    })
    (in1, in2) match {
        case (x: UInt, y: UInt) => io.out := x + y
        case (x: SInt, y: SInt) => io.out := x + y
        case _ => throw new Exception("I give up!")
    }
}
println(getVerilog(dut = new ConstantSum(3.U, 4.U)))
println(getVerilog(dut = new ConstantSum(-3.S, 4.S)))
println(getVerilog(dut = new ConstantSum(3.U, 4.S)))


It is good to remember that Chisel types generally should not be value matched.
Scala's match executes during circuit elaboration, but what you probably want is a post-elaboration comparison.
The following gives a syntax error:

In [ ]:
class InputIsZero extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(16.W))
        val out = Output(Bool())
    })
    io.out := (io.in match {
        // note that case 0.U is an error
        case (0.U) => true.B
        case _   => false.B
    })
}
println(getVerilog(new InputIsZero))

---
# Type Safe Connections<a name="type-safe-connections"></a>

Chisel can check the type for many connections, including:
* Bool/UInt to Clock

For other types, Chisel will let you connect them, but may truncate/pad bits as appropriate.
* Bool/UInt to Bool/UInt
* Bundle to Bundle

In [ ]:
class Bundle1 extends Bundle {
  val a = UInt(8.W)
}

class Bundle2 extends Bundle1 {
  val b = UInt(16.W)
}

class BadTypeModule extends Module {
  val io = IO(new Bundle {
    val c  = Input(Clock())
    val in = Input(UInt(2.W))
    val out = Output(Bool())

    val bundleIn = Input(new Bundle2)
    val bundleOut = Output(new Bundle1)
  })
  
  //io.out := io.c // won't work due to different types

  // Okay, but Chisel will truncate the input width to 1 to match the output.
//   io.out := io.in

//   // Compiles; Chisel will connect the common subelements of the two Bundles (in this case, 'a').
//   io.bundleOut := io.bundleIn
}

println(getVerilog(new BadTypeModule))

---
# Type Generics<a name="type-generics"></a>
Scala's generic types (also known as polymorphism) is very complicated, especially when coupling it with inheritance.

This section will just get your toes wet; to understand more, check out [this tutorial](https://twitter.github.io/scala_school/type-basics.html).

Classes can be polymorphic in their types. One good example is sequences, which require knowing their contained type.

In [ ]:
val seq1 = Seq("1", "2", "3") // Type is Seq[String]
val seq2 = Seq(1, 2, 3)       // Type is Seq[Int]
val seq3 = Seq(1, "2", true)  // Type is Seq[Any]

Sometimes, the Scala compiler needs help determining a polymorphic type, which requires the user to explicitly put the type:

In [ ]:
//val default = Seq() // Error!
val default = Seq[String]() // User must tell compiler that default is of type Seq[String]
Seq(1, "2", true).foldLeft(default){ (strings, next) =>
    next match {
        case s: String => strings ++ Seq(s)
        case _ => strings
    }
}

Functions can also be polymorphic in their input or output types. The following example defines a function that times how long it takes to run a block of code. It is parameterized based on the return type of the block of code. *Note that the `=> T` syntax encodes an anonymous function that does not have an argument list, e.g. `{ ... }` versus `{ x => ... }`.*

In [ ]:
def time[T](block: => T): T = {
    val t0 = System.nanoTime()
    val result = block
    val t1 = System.nanoTime()
    val timeMillis = (t1 - t0) / 1000000.0
    println(s"Block took $timeMillis milliseconds!")
    result
}

// Adds 1 through a million
val int = time { (1 to 1000000).reduce(_ + _) }
println(s"Add 1 through a million is $int")

// Finds the largest number under a million that, in hex, contains "beef"
val string = time {
    (1 to 1000000).map(_.toHexString).filter(_.contains("beef")).last
}
println(s"The largest number under a million that has beef: $string")

## Chisel Type Hierarchy
To write type generic code with Chisel, it is helpful to know a bit about the type hierarchy of Chisel.

`chisel3.Data` is the base class for Chisel hardware types.
`UInt`, `SInt`, `Vec`, `Bundle`, etc. are all instances of `Data`.
`Data` can be used in IOs and supports `:=`, wires, regs, etc.

Registers are a good example of polymorphic code in Chisel.
Look at the implementation of `RegEnable` (a register with a `Bool` enable signal) [here](https://github.com/freechipsproject/chisel3/blob/v3.0.0/src/main/scala/chisel3/util/Reg.scala#L10).
The apply function is templated for `[T <: Data]`, which means `RegEnable` will work for all Chisel hardware types.

Some operations are only defined on subtypes of `Bits`, for example `+`.
This is why you can add `UInt`s or `SInt`s but not `Bundle`s or `Vec`s.

<span style="color:blue">**Example: Type Generic ShiftRegister**<a name="type-generic-shift-register"></a></span><br>
In Scala, objects and functions aren't the only things we can treat as parameters.
We can also treat types as parameters.

We usually need to provide a type constraint.
In this case, we want to be able to put objects in a bundle, connect (:=) them, and create registers with them (RegNext).
These operations cannot be done on arbitrary objects; for example wire := 3 is illegal because 3 is a Scala Int, not a Chisel UInt.
If we use a type constraint to say that type T is a subclass of Data, then we can use := on any objects of type T because := is defined for all Data.

Here is an implementation of a shift register that take types as a parameter.
*gen* is an argument of type T that tells what width to use, for example new ShiftRegister(UInt(4.W)) is a shift register for 4-bit UInts.
*gen* also allows the Scala compiler to infer the type T- you can write new ShiftRegister[UInt](UInt(4.W)) if you want to to be more specific, but the Scala compiler is smart enough to figure it out if you leave out the [UInt].

In [ ]:
class ShiftRegisterIO[T <: Data](gen: T, n: Int) extends Bundle {
    require (n >= 0, "Shift register must have non-negative shift")
    
    val in = Input(gen)
    val out = Output(Vec(n + 1, gen)) // + 1 because in is included in out
    override def cloneType: this.type = (new ShiftRegisterIO(gen, n)).asInstanceOf[this.type]
}

class ShiftRegister[T <: Data](gen: T, n: Int) extends Module {
    val io = IO(new ShiftRegisterIO(gen, n))
    
    io.out.foldLeft(io.in) { case (in, out) =>
        out := in
        RegNext(in)
    }
}

visualize(() => new ShiftRegister(SInt(6.W), 3))
test(new ShiftRegister(SInt(6.W), 3)) { c => 
    println(s"Testing ShiftRegister of type ${c.io.in} and depth ${c.io.out.length}")
    for (i <- 0 until 10) {
        c.io.in.poke(i.S) // magic literal creation
        println(s"$i: ${c.io.out.indices.map { index => c.io.out(index).peek().litValue} }")
        c.clock.step(1)
    }}

We generally recommend avoiding to use inheritance with type generics.
It can be very tricky to do properly and can get frustrating quickly.

## Type Generics with Typeclasses

The example above was limited to simple operations that could be performed on any instance of `Data` such as `:=` or `RegNext()`.
When generating DSP circuits, we would like to do mathematical operations like addition and multiplication.
The `dsptools` library provides tools for writing type parameterized DSP generators.

Here is an example of writing a multiply-accumulate module.
It can be used to generate a multiply-accumulate (MAC) for `FixedPoint`, `SInt`, or even `DspComplex[T]` (the complex number type provided by `dsptools`).
The syntax of the type bound is a little different because `dsptools` uses typeclasses.
They are beyond the scope of this notebook.
Read the `dsptools` readme and documentation for more information on using typeclasses.

`T <: Data : Ring` means that `T` is a subtype of `Data` and is also a `Ring` .
`Ring` is defined in `dsptools` as a number with `+` and `*` (among other operations).

_An alternative to `Ring` would be `Real`, but that would not allow us to make a MAC for `DspComplex()` because complex numbers are not `Real`._



In [ ]:
import chisel3.experimental._
import dsptools.numbers._

class Mac[T <: Data : Ring](genIn : T, genOut: T) extends Module {
    val io = IO(new Bundle {
        val a = Input(genIn)
        val b = Input(genIn)
        val c = Input(genIn)
        val out = Output(genOut)
    })
    io.out := io.a * io.b + io.c
}

println(getVerilog(new Mac(UInt(4.W), UInt(6.W)) ))
println(getVerilog(new Mac(SInt(4.W), SInt(6.W)) ))
println(getVerilog(new Mac(FixedPoint(4.W, 3.BP), FixedPoint(6.W, 4.BP))))
